<a href="https://colab.research.google.com/github/yallapragada/text_similarity/blob/master/text_similarity_doc2vec_annoy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
!pip install stop_words
from stop_words import get_stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=28acd46d4d000fbffc2c98e5450dd9e4cc778fd1b9127cd8ea21f80d70e91999
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [5]:
from google.colab import files
uploaded = files.upload()

Saving arxivData.json to arxivData.json


In [9]:
import io
import pandas as pd
df = pd.read_json(io.StringIO(uploaded['arxivData.json'].decode('ISO-8859-1')))
df.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017


In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [0]:
stop_words = get_stop_words('en')

In [0]:
def get_tagged_document(text, id):
  doc = nlp(text)
  words = [token.text for token in doc if token.text not in stop_words and len(token.text) > 2]
  tagged_document = TaggedDocument(words, tags=[id])
  return tagged_document

In [0]:
tagged_documents = []
for i,row in df.iterrows():
  if i % 500 == 0:
    print(i)
  text = row['title'] + ' ' + row['summary']
  text = text.lower()
  id = row['id']
  tagged_document = get_tagged_document(text=text, id=id)
  tagged_documents.append(tagged_document)

In [0]:
tagged_documents = [tagged_document for tagged_document in tagged_documents if (len(tagged_document.words) > 15)]

In [48]:
len(tagged_documents)

40953

In [0]:
model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, workers=4, epochs=10)

In [21]:
!pip install annoy
from gensim.similarities.index import AnnoyIndexer

     |████████████████████████████████| 645kB 4.7MB/s 
  Created wheel for annoy: filename=annoy-1.16.0-cp36-cp36m-linux_x86_64.whl size=309746 sha256=4f4a25659f2b158e127088a1578bbcf29c77308a41b2f1f43fca4b2778d1db28
  Stored in directory: /root/.cache/pip/wheels/39/34/9c/8d8eb3abce9375bade7b42bca387700b5f3aa18414d27272f0
Successfully built annoy


In [50]:
annoy_index = AnnoyIndexer(model, 30)

/usr/local/lib/python3.6/dist-packages/gensim/similarities/index.py:180: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


In [25]:
len(df)

41000

In [0]:
test_doc = '''
In this paper, we consider the task of
text categorization as a graph classification problem. By representing textual documents as graph-of-words instead of historical n-gram bag-of-words, we extract
more discriminative features that correspond to long-distance n-grams through
frequent subgraph mining. Moreover, by
capitalizing on the concept of k-core, we
reduce the graph representation to its densest part – its main core – speeding up the
feature extraction step for little to no cost
in prediction performances. Experiments
on four standard text classification datasets
show statistically significant higher accuracy and macro-averaged F1-score compared to baseline approaches.
'''

In [0]:
test_doc = test_doc.replace('\n', ' ').lower()

In [28]:
test_doc

' in this paper, we consider the task of text categorization as a graph classification problem. by representing textual documents as graph-of-words instead of historical n-gram bag-of-words, we extract more discriminative features that correspond to long-distance n-grams through frequent subgraph mining. moreover, by capitalizing on the concept of k-core, we reduce the graph representation to its densest part – its main core – speeding up the feature extraction step for little to no cost in prediction performances. experiments on four standard text classification datasets show statistically significant higher accuracy and macro-averaged f1-score compared to baseline approaches. '

In [0]:
doc = nlp(test_doc)
words = [token.text for token in doc if token.text not in stop_words and len(token.text) > 2]

In [0]:
vector = model.infer_vector(words)

In [60]:
neighbors = model.most_similar([vector], topn=5, indexer=annoy_index)
for neighbor in neighbors:
    print(df.loc[df['id'] == neighbor[0], 'summary'].iloc[0])

Graph classification is a problem with practical applications in many
different domains. Most of the existing methods take the entire graph into
account when calculating graph features. In a graphlet-based approach, for
instance, the entire graph is processed to get the total count of different
graphlets or sub-graphs. In the real-world, however, graphs can be both large
and noisy with discriminative patterns confined to certain regions in the graph
only. In this work, we study the problem of attentional processing for graph
classification. The use of attention allows us to focus on small but
informative parts of the graph, avoiding noise in the rest of the graph. We
present a novel RNN model, called the Graph Attention Model (GAM), that
processes only a portion of the graph by adaptively selecting a sequence of
"interesting" nodes. The model is equipped with an external memory component
which allows it to integrate information gathered from different parts of the
graph. We demonstrate

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
